# 1. Reading Data 

In [2]:
import pandas as pd
import os
import glob
import os
import random
import pandas as pd
import numpy as np 
%matplotlib inline
import matplotlib
from fastai.imports import *
from fastai.structured import *
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import Dense
from keras import models
from sklearn.ensemble import IsolationForest
from keras import optimizers
import itertools
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import math
from datetime import datetime, timedelta
from fastai.imports import *
from fastai.structured import *
os.chdir("/scratch/rk9cx/conn_log_labelled_runtime/labelled_2019-01-10//")

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#random sampling
filename = "merged_2019-01-10.csv"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 1000000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list

In [4]:
#preprocessing data
df = pd.read_csv(filename, skiprows=skip)
df = df.drop(df.columns[0], axis=1)
df = pd.concat([df, df['history'].str.join('|').str.get_dummies()], axis = 1)
df = df.drop(["honeypot","blacklist","whitelist"], axis=1)
df = pd.concat([df, pd.get_dummies(df.conn_state.apply(pd.Series), prefix="", prefix_sep="")], axis = 1)
df = df.drop(["conn_state"], axis=1)

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,3,5,11,12,13,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df["duration"] = df["duration"].replace('-', 0)
df["src_bytes"] = df["src_bytes"].replace('-', 0)
df["dest_bytes"] = df["dest_bytes"].replace('-', 0)

In [2]:
sample = df.sample(100)
df.T.head(50)

NameError: name 'df' is not defined

# 2. Word2Vec Implementation

In [7]:
import os
import pandas as pd
import nltk
import gensim
nltk.download('punkt')
from gensim import corpora, models, similarities

[nltk_data] Downloading package punkt to /home/rk9cx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
x=df['history'].values.tolist()

In [9]:
tok_corp= [nltk.word_tokenize(sent) for sent in x]

In [ ]:
tok_corp = []
for sent in x:
    print(sent)
    tok_corp.append(nltk.word_tokenize(sent))

In [11]:
model = gensim.models.Word2Vec(tok_corp, min_count=1, size = 5)

In [12]:
model[df["history"][0]]

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.06721, -0.01748,  0.04797,  0.07438, -0.06444], dtype=float32)

In [13]:
model["S"]

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.06721, -0.01748,  0.04797,  0.07438, -0.06444], dtype=float32)

In [14]:
def word_vector(ele):
    return model[ele]

In [21]:
datafra = df.head(10000)

In [22]:
allword_vecs = pd.DataFrame()
count = 0
for index,row in datafra.iterrows():
    A = model[row["history"]]
    word_vecs = pd.DataFrame(A).T
    word_vecs["history"] = row["history"]
    allword_vecs = allword_vecs.append(word_vecs)
    count = count +1
    if count == 50000:
        print("50%")
    elif count == 100000:
        print("100%")

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [24]:
finaldf = pd.merge(datafra,allword_vecs, on = 'history')

In [25]:
finaldf.head()

,ts,src_ip,src_port,dest_ip,dest_port,duration,src_bytes,dest_bytes,history,src_pkts,...,S2,S3,SF,SH,SHR,0,1,2,3,4
0,1.5471e+09,194.28.115.245,47821,3.110.16.42,9090,0,0,0,S,1,...,0,0,0,0,0,0.067209,-0.017477,0.047971,0.074376,-0.064439
1,1.5471e+09,194.28.115.245,47821,3.110.16.42,9090,0,0,0,S,1,...,0,0,0,0,0,0.067209,-0.017477,0.047971,0.074376,-0.064439
2,1.5471e+09,194.28.115.245,47821,3.110.16.42,9090,0,0,0,S,1,...,0,0,0,0,0,0.067209,-0.017477,0.047971,0.074376,-0.064439
3,1.5471e+09,194.28.115.245,47821,3.110.16.42,9090,0,0,0,S,1,...,0,0,0,0,0,0.067209,-0.017477,0.047971,0.074376,-0.064439
4,1.5471e+09,194.28.115.245,47821,3.110.16.42,9090,0,0,0,S,1,...,0,0,0,0,0,0.067209,-0.017477,0.047971,0.074376,-0.064439


In [ ]:
finaldf.to_csv("word2vec.csv", index = False)

In [99]:
datafra["history"] = datafra["history"].astype(str)
allword_vecs["history"] = allword_vecs["history"].astype(str)

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [120]:
allword_vecs.head()

,0,1,2,3,4,history
0,-0.094483,0.080201,0.083023,-0.024449,-0.023619,S
0,-0.094483,0.080201,0.083023,-0.024449,-0.023619,S
0,-0.094483,0.080201,0.083023,-0.024449,-0.023619,S
0,-0.094483,0.080201,0.083023,-0.024449,-0.023619,S
0,-0.094483,0.080201,0.083023,-0.024449,-0.023619,S


In [ ]:
mergedf = pd.merge(datafra,allword_vecs, how='left', on = ['history'])
mergedf.head()

In [110]:
mergedf.shape

(100000, 50)

In [69]:
uni_wordvecs = allword_vecs.drop_duplicates(keep = False)

In [73]:
finaldf.columns

Index([        'ts',     'src_ip',   'src_port',    'dest_ip',  'dest_port',
         'duration',  'src_bytes', 'dest_bytes',    'history',   'src_pkts',
        'dest_pkts',      'local', 'src_ip_ext',      'label',          'A',
                'D',          'F',          'H',          'Q',          'R',
                'S',          'T',          '^',          'a',          'd',
                'f',          'h',          'i',          'q',          'r',
                's',          't',        'OTH',        'REJ',       'RSTO',
           'RSTOS0',       'RSTR',      'RSTRH',         'S0',         'S1',
               'S2',         'S3',         'SF',         'SH',        'SHR',
                  0,            1,            2,            3,            4],
      dtype='object')

In [67]:
#!/usr/bin/env python
import psutil
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 270373085184,
 'available': 183288000512,
 'percent': 32.2,
 'used': 69869256704,
 'free': 107783045120,
 'active': 109342785536,
 'inactive': 38526697472,
 'buffers': 0,
 'cached': 92720783360,
 'shared': 16310538240,
 'slab': 11334295552}

# 3. Preprocessing and Train-Test Splitting

In [ ]:
def class_balancer(df):
    if df.label.value_counts()[0] > df.label.value_counts()[1]:
        mal = df.loc[df['label'] == 1]
        ben = df.loc[df['label'] == 0].sample(df.label.value_counts()[1])
        mdf = pd.concat([mal,ben], axis = 0)
        return mdf
    if df.label.value_counts()[1] > df.label.value_counts()[0]:
        mal = df.loc[df['label'] == 1].sample(df.label.value_counts()[0])
        ben = df.loc[df['label'] == 0]
        mdf = pd.concat([mal,ben], axis = 0)
        return mdf

In [ ]:
finaldf.drop(["src_ip","dest_ip","src_ip_ext"], axis = 1, inplace=True)

In [37]:
finaldf["duration"] = finaldf["duration"].astype('float')
finaldf["src_bytes"] = finaldf["src_bytes"].astype('float')
finaldf["dest_bytes"] = finaldf["dest_bytes"].astype('float')
finaldf["src_pkts"] = finaldf["src_pkts"].astype('int')
finaldf["dest_pkts"] = finaldf["dest_pkts"].astype('int')
finaldf["local"] = finaldf["local"].astype('int')
finaldf["label"] = finaldf["label"].astype('int')

In [40]:
    finaldf["src_bytes"] = finaldf["src_bytes"].astype(float)
    finaldf["src_bytes"] = np.log((finaldf["src_bytes"].astype('int'))+0.001)
    finaldf["dest_bytes"] = finaldf["dest_bytes"].astype(float)
    finaldf["dest_bytes"] = np.log((finaldf["dest_bytes"].astype('int'))+0.001)
    finaldf["src_pkts"] = np.log((finaldf["src_pkts"].astype('int'))+0.001)
    finaldf["dest_pkts"] = np.log((finaldf["dest_pkts"].astype('int'))+0.001)
    finaldf['ts'] = pd.to_datetime(finaldf['ts'],unit='s')
    finaldf["src_port"] = finaldf["src_port"].astype('category')
    finaldf["dest_port"] = finaldf["dest_port"].astype('category')
    finaldf["src_port"] = finaldf["src_port"].cat.codes
    finaldf["dest_port"] = finaldf["dest_port"].cat.codes

In [ ]:
finaldf["duration"] = finaldf["duration"].astype(float)
finaldf["duration"] = np.log((finaldf["duration"].astype('int'))+0.001)
add_datepart(finaldf, 'ts')

In [1]:
finaldf = finaldf.sort_values(by=['tsElapsed'], ascending=True)

NameError: name 'finaldf' is not defined

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
# split point: length of dataset minus validation set size.
split_point = len(X)-validation

# split X
X_train, X_test = split_vals(X, split_point)

# split y
y_train, y_test = split_vals(Y, split_point)

In [ ]:
# 4. Modelling